This Jupyter Notebook will be used for my Coursera Capstone Project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


Scraping the wikipedia page-Web Scraping using Beautiful Soup
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Mon, 27 Apr 2020 02:16:38 GMT', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Server': 'ATS/8.0.7', 'Content-Type': 'text/html; charset=UTF-8', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Last-Modified': 'Sat, 18 Apr 2020 18:27:33 GMT', 'Content-Encoding': 'gzip', 'Age': '54563', 'X-Cache': 'cp5008 miss, cp5010 hit/35', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=27-Apr-2020;Path=/;HttpOnly;secure;Expires=Fri, 29 May 2020 12:00:00 GMT, WMF-Last-Access-Global=27-Apr-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Fri, 29 May 2020 12:00:00 GMT, GeoIP=IN:TG:Hyderabad:17.38:78.46:v4; Path=/; secure; Domain=.wikipedia.org', 'X-Client-IP': '2401:4900:4829

In [5]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

#lst

Converting into a dataframe

In [6]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes

(103, 3)


custom groupby / agg to merge Neighborhoods

groupby PostalCode, keep the first Borough and join() Neighborhoods

In [7]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

will check if 'M5A' example is done correctly

In [8]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,Regent Park / Harbourfront


In [9]:
df = df.replace({'/': ','}, regex=True)

In [10]:
df.shape

(103, 3)

Reading the df object into a csv file

In [11]:
df.to_csv('/home/user/Courses/Machine Learning/Data Science Coursera/Applied Data Science Capstone/postalcodes.csv')

will read geo data from a CSV
need to rename a column so the merge (below) will work

In [12]:
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov..."
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."


Here we take the dataset containing the latitude and longitude values,so first we import this dataset into the df2 variable

In [22]:
df2=pd.read_csv('/home/user/Courses/Machine Learning/Data Science Coursera/Applied Data Science Capstone/Files/Geospatial_Coordinates.csv')

In [23]:
df2

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Take the latitude values into a and the longitude values into b

In [24]:
a=df2['Latitude']
b=df2['Longitude']

Now for the old dataset(df) add 2 columns and assign these latitude and longitude values so now this new dataframe will contain 5 columns(along with the latitude and the longitude values)

In [25]:
df['Latidude']=a
df['Longitude']=b

In [26]:
df

,PostalCode,Borough,Neighborhood,Latidude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437


In [27]:
df.to_csv('/home/user/Courses/Machine Learning/Data Science Coursera/Applied Data Science Capstone/Files/new.csv')

In [28]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))


/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


In [32]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latidude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto